In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
train_dir = "../Dataset/train/"


test_dir = "../Dataset/test/"




train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    shear_range=0.2,  
    zoom_range=0.2,  
    horizontal_flip=True  
)


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), 
    batch_size=32,
    class_mode='binary'  
)


validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [3]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=10,  
    epochs=100,  
    validation_data=validation_generator,
    validation_steps=50  
)


Epoch 1/100
10/10 [==============================] - 23s 2s/step - loss: 0.7474 - accuracy: 0.5344 - val_loss: 0.6768 - val_accuracy: 0.5455
Epoch 2/100
10/10 [==============================] - 10s 1s/step - loss: 0.6702 - accuracy: 0.5594
Epoch 3/100
10/10 [==============================] - 8s 826ms/step - loss: 0.6267 - accuracy: 0.6512
Epoch 4/100
10/10 [==============================] - 9s 835ms/step - loss: 0.6136 - accuracy: 0.7063
Epoch 5/100
10/10 [==============================] - 8s 763ms/step - loss: 0.6393 - accuracy: 0.5750
Epoch 6/100
10/10 [==============================] - 7s 731ms/step - loss: 0.6316 - accuracy: 0.6750
Epoch 7/100
10/10 [==============================] - 7s 678ms/step - loss: 0.5080 - accuracy: 0.7281
Epoch 8/100
10/10 [==============================] - 8s 778ms/step - loss: 0.5210 - accuracy: 0.7719
Epoch 9/100
10/10 [==============================] - 7s 691ms/step - loss: 0.4969 - accuracy: 0.7688
Epoch 10/100
10/10 [==============================] -

In [9]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, batch_size=64)
print("Accuracy:", accuracy)

# Make predictions on the test data
predictions = model.predict(validation_generator)
predicted_classes = (predictions > 0.5).astype(int)

# Calculate precision, recall, and f1-score
report = classification_report(validation_generator.classes, predicted_classes, target_names=validation_generator.class_indices)
print(report)

# Calculate confusion matrix
confusion_mat = confusion_matrix(validation_generator.classes, predicted_classes)
print("Confusion Matrix:")
print(confusion_mat)

21/21 [==============================] - 2s 84ms/step - loss: 0.3754 - accuracy: 0.7924
Accuracy: 0.7924242615699768
21/21 [==============================] - 2s 86ms/step
              precision    recall  f1-score   support

      benign       0.52      0.33      0.40       360
   malignant       0.44      0.62      0.51       300

    accuracy                           0.47       660
   macro avg       0.48      0.48      0.46       660
weighted avg       0.48      0.47      0.45       660

Confusion Matrix:
[[120 240]
 [113 187]]
